<hr />

# G2M insight for Cab Investment firm
<hr />
<hr />
<br />


## Problem Statement
<hr />

XYZ is a private firm in US. Due to remarkable growth in the Cab Industry in last few years and multiple key players in the market, it is planning for an investment in Cab industry and as per their Go-to-Market(G2M) strategy they want to understand the market before taking final decision.

You have been provided with multiple data sets that contains information on 2 cab companies. Each file (data set) provided represents different aspects of the customer profile. XYZ is interested in using your actionable insights to help them identify the right company to make their investment.

<br />
<br />


## Data Understanding
<hr />

You have been provided 4 individual data sets. Time period of data is from 31/01/2016 to 31/12/2018.

Below are the list of datasets which are provided for the analysis:

<ol>
    <li> <b> Cab_Data.csv : </b> this file includes details of transaction for 2 cab companies </li>
    <li> <b> Customer_ID.csv : </b> this is a mapping table that contains a unique identifier which links the customer’s demographic details </li>
    <li> <b> Transaction_ID.csv : </b> this is a mapping table that contains transaction to customer mapping and payment mode </li>
    <li> <b> City.csv : </b> this file contains list of US cities, their population and number of cab users </li>
</ol>

<br />


In [7]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')


In [2]:
!pip install fuzzywuzzy

In [8]:
import fuzzywuzzy
from fuzzywuzzy import process
import chardet


Let us import the datasets

In [9]:
cab_df = pd.read_csv('DataSets/Cab_Data.csv') # cab details
cust_df = pd.read_csv('DataSets/Customer_ID.csv') # customer details
trans_df = pd.read_csv('DataSets/Transaction_ID.csv') # transaction details
city_df = pd.read_csv('DataSets/City.csv') # city details


Let us analyse the 4 datasets to find the following:
1. Shape
2. Columns and their datatypes
3. Head / snapshot of the dataset
4. Unique info
5. Other statistics of numerical columns


### cab_df

In [10]:
cab_df.shape

(359392, 7)

In [11]:
cab_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 359392 entries, 0 to 359391
Data columns (total 7 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   Transaction ID  359392 non-null  int64  
 1   Date of Travel  359392 non-null  int64  
 2   Company         359392 non-null  object 
 3   City            359392 non-null  object 
 4   KM Travelled    359392 non-null  float64
 5   Price Charged   359392 non-null  float64
 6   Cost of Trip    359392 non-null  float64
dtypes: float64(3), int64(2), object(2)
memory usage: 19.2+ MB


In [ ]:
cab_df.describe()

In [ ]:
cab_df.head()

In [ ]:
print(cab_df.Company.value_counts(), '\n')
print(cab_df.City.value_counts(), '\n')
print(cab_df['Date of Travel'].value_counts())


<br />

### cust_df

In [ ]:
cust_df.shape

In [ ]:
cust_df.info()

In [ ]:
cust_df.describe()

In [ ]:
cust_df.head()

<br />

### trans_df

In [ ]:
trans_df.shape

In [ ]:
trans_df.info()

In [ ]:
trans_df.describe()

In [ ]:
trans_df.head()

In [ ]:
trans_df.Payment_Mode.value_counts()

<br />

### city_df

In [ ]:
city_df.shape

In [ ]:
city_df.info()

In [ ]:
city_df.describe()

In [ ]:
city_df.head()

In [ ]:
city_df.City.value_counts()

<br />
<br />

## Data Cleaning
<hr />

<ol>
    <li> Check for null values </li>
    <li> Check for duplicate entries </li>
    <li> Check names of columns </li>
    <li> Check datatype and values in columns </li>
    <li> Check unique values in columns </li>
    <li> Merge dataframes and drop columns not necessary for analysis </li>
    <li> Derive features, if necessary </li>
    <li> Check for outliers </li>
    <li> Sanity checks </li>
</ol>

<br />
<br />


<br />

### 1. Check for null values
<hr />

In [ ]:
print('Percentage of null values in:')
print('   cab_df: {}%\n   cust_df: {}%\n   trans_df: {}%\n   city_df: {}%'.format(
    round(cab_df.isnull().sum().sum() / cab_df.shape[0] * 100, 2),
    round(cust_df.isnull().sum().sum() / cust_df.shape[0] * 100, 2),
    round(trans_df.isnull().sum().sum() / trans_df.shape[0] * 100, 2),
    round(city_df.isnull().sum().sum() / city_df.shape[0] * 100, 2)
))


<br />
<br />

### 2. Check for duplicate entries
<hr />


In [ ]:
print('No of duplicate entries in: ')
print('   cab_df: {}\n   cust_df: {}\n   trans_df: {}\n   city_df:{}'.format(
    cab_df.duplicated().sum(), cust_df.duplicated().sum(), trans_df.duplicated().sum(), city_df.duplicated().sum()
))


<br />
<br />

### 3. Check names of the columns
<hr/>


In [ ]:
dataframes = {'cab_df': cab_df, 'cust_df': cust_df, 'trans_df': trans_df, 'city_df': city_df}

for df_name, df in dataframes.items():
    print(df_name, '\n', '='*25, '\n')
    print(df.info(), '\n')
    

Let us make the column names follow <i>snake_case</i>, for ease of use.

Let us also change <b>Income (USD/Month)</b> of cust_df to <b>income_usd_per_month</b>.


In [ ]:
for df_name, df in dataframes.items():
    if df_name == 'cust_df':
        df.rename(columns={'Income (USD/Month)':'income_usd_per_month'}, inplace=True)
    cols = ['_'.join(col.lower().split()) for col in df.columns]
    df.columns = cols
    # check if the change was made
    print(df_name, '\n', '='*10, '\n')
    print(df.columns, '\n')
    

<br />
<br />

### 4. Check datatype of columns
<hr />


In [ ]:

for df_name, df in dataframes.items():
    print(df_name, '\n', '='*25, '\n')
    print(df.info(), '\n')
    print(df.head(3),'\n', '-'*100, '\n')
    

### <i>Observations</i>
------------------------------
1. cab_df: The <b>date_of_travel</b> column is of type </i>integer</i>. It should be <i>datetime</i>. Moreover, it has erronous values
2. city_df: The <b>population</b> and <b>users</b> columns are of type <i>object</i>. It should be <i>integer</i>
</b>




In [ ]:
# check unique values in population and users column in city_df
print(city_df.population.value_counts())
print("\n\n", city_df.users.value_counts())


We can convert both these columns from type <i>object</i> to type <i>int</i> by removing the commas.

In [ ]:
# convert population and users to type int
print("Before conversion: ", city_df.info(), "\n")
print(city_df.head(), "\n\n\n")

city_df.population = city_df.population.apply(lambda pop: int(pop.strip().replace(',','')))
city_df.users = city_df.users.apply(lambda us: int(us.strip().replace(',','')))

print("After conversion: ", city_df.info(), "\n")
print(city_df.head(), "\n\n\n")


<br />
<br />

### 4. Check unique values in columns
<hr />


In [ ]:
for df_name, df in dataframes.items():
    print(df_name, "   ", df.shape)
    print('='*25, '\n')
    for col in df.columns:
        print(col)
        print('-'*15)
        print('No of unique values: ', df[col].nunique())
        print('Percentage of unique values: ', round(df[col].nunique() / df.shape[0] * 100, 2), '%')
        print(df[col].value_counts(), '\n')
    print('\n\n')
    

### <i>Observations</i>
------------------------------
1. ### cab_df:
   ----------
    - <b>transaction_id</b> has 100% unique values. We can drop this column as it will add no value to our analysis.
    - <b>date_of_travel</b> has date stored as excel serial date. Let us convert it to datetime.
    - <b>company</b> has on only 2 unique values: Yellow cab and Pink cab. This column segments the whole data to 2.
    - <b>city</b> also has considerable amount of unique values, 19.
    - <b>km_travelled</b> has 0.24% unique values. That is 874 out of the 359392. We can have more unique values by rounding off the floating value to nearest integer.
    - We can similarly round of values of <b>price_charged</b> and <b>cost_of_trip</b>.
    
    <br />
    
2. ### cust_df
   ----------
   - <b>customer_id</b> has 100% unique values. We can drop this column as it will add no value to our analysis.
   - <b>gender</b> has 2 unique values: Male and Female
   - <b>age</b> has 0.1% unique values. That is 48 out of the total 49171 values are unique. We can have more unique values by giving in age groups, rather than age just like that.
   - <b>income_usd_per_month</b> has about 50% unique values. We can have more unique values by rounding of this value.
   
   <br />
   
3. ### trans_df
   ----------
   - <b>transaction_id</b> has 100% unique values. We can drop this column as it will add no value to our analysis.
   - <b>customer_id</b> has all the values in the customer_id column of cust_df.
   - <b>payment_mode</b> has 2 unique values: Cash and Card
   
   <br />
   
4. ### city_df
   ----------
   - All the columns are of significance here.
   

<br />

   
There are some columns that can be dropped. But, we will do the same only after we rectify the errors found in some of them followed by merging all the dataframes into a master dataframe.

   
<br />
<br />


In [ ]:
# convert date_of_travel to type datetime
import xlrd

print("Before conversion: \n")
print(cab_df.info(), "\n")
print(cab_df.head(), "\n\n\n")

cab_df.date_of_travel = cab_df.date_of_travel.apply(lambda d: xlrd.xldate_as_datetime(d, 0).date())
cab_df.date_of_travel = pd.to_datetime(cab_df.date_of_travel)

print("After conversion: \n")
print(cab_df.info(), "\n")
print(cab_df.head())


In [ ]:
# # round values in km_travelled, price_charged and cost_of_trip
# cols = ['km_travelled', 'price_charged', 'cost_of_trip']
# print("Before rounding values: \n")
# for col in cols:
#         print(col)
#         print('-'*15)
#         print('No of unique values: ', cab_df[col].nunique())
#         print('Percentage of unique values: ', round(cab_df[col].nunique() / cab_df.shape[0] * 100, 2), '%')
#         print(cab_df[col].value_counts(), '\n')
# print("\n\n\n")

# for col in cols:
#     cab_df[col] = cab_df[col].apply(lambda v: round(v))

# print("After rounding values: \n")
# for col in cols:
#         print(col)
#         print('-'*15)
#         print('No of unique values: ', cab_df[col].nunique())
#         print('Percentage of unique values: ', round(cab_df[col].nunique() / cab_df.shape[0] * 100, 2), '%')
#         print(cab_df[col].value_counts(), '\n')
# print("\n\n")

Let us use fuzzy logic to check for same entries with different spelling etc.. in the <b>city</b> column of <b>city_df</b>.

In [ ]:
cab_df.head()

In [ ]:
# get the top 5 closest matches to each fo the city names in both cab_df and city_df
print("cab_df")
print("-"*25)
for city in cab_df.city.unique():
    matches = fuzzywuzzy.process.extract(city, cab_df.city.unique(), limit=3, scorer=fuzzywuzzy.fuzz.token_sort_ratio)
    print(city, ":  --->  ", matches)

    
print("\n\ncity_df")
print("-"*25)
for city in city_df.city.unique():
    matches = fuzzywuzzy.process.extract(city, city_df.city.unique(), limit=3, scorer=fuzzywuzzy.fuzz.token_sort_ratio)
    print(city, ":  --->  ", matches)
    

### <i>Observations</i>
<hr />

There are no city entries with erros. All are unique.

<br />


<br />
<br />

### 6. Merge dataframes and drop columns not necessary for analysis
<hr />

Now, let us merge all the dataframes together to get a meaningfull master dataframe.

In [ ]:
for df_name, df in dataframes.items():
    print(df_name, "   ---   ", df.shape)
    print('-'*25, "\n")
    print(df.head(), "\n\n")
    

In [ ]:
# Merge cab_df and trans_df  on transaction_id
master_df = pd.merge(cab_df, trans_df, on='transaction_id', how='inner')
print(master_df.shape)
master_df.head()


In [ ]:
# Merge cust_df and master_df on cust_id
master_df = pd.merge(master_df, cust_df, on='customer_id', how='inner')
print(master_df.shape)
master_df.head()


In [ ]:
# Merge city_df and master_df on city
master_df = pd.merge(master_df, city_df, on='city', how='inner')
print(master_df.shape)
master_df.head()


In [ ]:
# Check for null values and duplicate entries
print('Percentage of null values in master_df : ', round(master_df.isnull().sum().sum() / master_df.shape[0] * 100, 2), '%')
print('No of duplicate entries in master_df : ', master_df.duplicated().sum(), "\n\n\n")


Now, let us see if there are any columns that can be dropped

In [ ]:
master_df.head()


In [ ]:
master_df.shape


In [ ]:
master_df.info()


In [ ]:
master_df.describe()


In [ ]:
for col in master_df.columns:
    print(col)
    print('-'*25)
    print("No of unique values in {}: {}".format(col, master_df[col].nunique()))
    print("Percentage of unique values in {}: {}%\n".format(col, round(master_df[col].nunique()/master_df.shape[0]*100), 3))
    print(master_df[col].value_counts(), "\n\n")
    

### <i>Observations</i>
------------------------------
1. <b>transaction_id</b> has 100% unique values. SO we can drop this column.
2. <b>customer_id</b> is also a similar id column. But we might need this column because we can segment the dfataframe based on the demographics of users of the cabs.



In [ ]:
# drop transaction_id
print("Before column drop: ", list(master_df.columns), "\n")

master_df.drop(columns='transaction_id', inplace=True)

print("After column drop: ", list(master_df.columns), "\n\n")


<br />
<br />

### 7. Derive features, if necessary
<hr />


Let us take a look at the columns to see if we can dervie some features out of it

In [ ]:
master_df.info()

In [ ]:
master_df.head()

### <i>Observations</i>
------------------------------
1. We can derive the following columns from date_of_travel:
    - <b>month_of_travel</b>
    - <b>day_of_travel</b>
    - <b>year_of_travel</b>
    - <b>is_weekend</b>
    - <b>weekday_of_travel</b>
    
    
2. We can also make another feature - age group from the age column
    
    

In [ ]:
# derive month_of_travel
master_df['month_of_travel'] = master_df.date_of_travel.dt.month
month_list = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
master_df.month_of_travel = master_df.month_of_travel.apply(lambda m: month_list[m-1])

# derive year_of_travel
master_df['year_of_travel'] = master_df.date_of_travel.dt.year

# dervive day_of_travel
master_df['day_of_travel'] = master_df.date_of_travel.dt.day

# derive weekday_of_travel
master_df['weekday_of_travel'] = master_df.date_of_travel.dt.weekday
weekdays = ['Monday', 'tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
master_df.weekday_of_travel = master_df.weekday_of_travel.apply(lambda d: weekdays[d])

# derive is_weekend - 0(Monday), ..., 4(Friday), 5(Staurday), 6(Sunday)
master_df['is_weekend'] = master_df.weekday_of_travel.apply(lambda day: 'Yes' if day in ['Saturday', 'Sunday'] else 'No')

# Finally drop the date_of_travel column since we have extracted all the features from it
master_df.drop(columns='date_of_travel', inplace=True)

master_df.head()


In [ ]:
# derive column - age group
master_df['age_group'] = pd.cut( master_df.age, 
                                bins=[0, 15, 19, 29, 39, 49, 59, 200], 
                                labels=['Kid', 'Teenager', '20s', '30s', '40s', '50s', 'Senior Citizen'])


In [ ]:
master_df.info()

In [ ]:

# unique values in columns in master_df
for col in master_df.columns:
    print(col)
    print('-'*25)
    print("No of unique values in {}: {}".format(col, master_df[col].nunique()))
    print("Percentage of unique values in {}: {}%\n".format(col, round(master_df[col].nunique()/master_df.shape[0]*100), 3))
    print(master_df[col].value_counts(), "\n\n")
    

<br />
<br />

### 8. Check for outliers
<hr />


In [ ]:
master_df.describe()


Let us check for outliers by plotting box plots for each of the numerical columns

In [ ]:
sns.set_style('darkgrid')


In [ ]:
fig, ax = plt.subplots(4,3, figsize=(20,20))

# select numeric columns only
numeric_cols = list(master_df.select_dtypes(include=np.number).columns)

# set title for subplots
fig.suptitle("Box plots for numeric columns to find outliers", fontsize=30)


for i, col in enumerate(numeric_cols):
    r = i // 3
    c = i % 3
    sns.boxplot(master_df[col], ax=ax[r][c])
    ax[r,c].set_title(col, fontsize=15)
    
ax[3,1].set_axis_off()
ax[3,2].set_axis_off()


### <i>Observations</i>
------------------------------
The only column that seems to have outliers is <b>price_charged</b>. Let us take a look at the same. Moreover, this column seems to be dependent on the <b>cost_of_trip</b> column, which on the other hand has no outliers.
   

In [ ]:
# check whether hypothesis that price_changed and cost_of_trip is correlated by plotting a heatmap
plt.figure(figsize=(10, 10))
sns.heatmap(master_df.corr(), cmap='Blues', annot=True, linewidth=0.5, fmt='.2f')


### <i>Observations</i>
------------------------------
Dark colour signifies high correlation(->1) and light colour signifies low correlation(->0).

For <b>price_changed</b> and <b>cost_of_trip</b> the correlation colour is dark blue, which corresponds to a correlation of ~0.86 (86%). This is a very high value. The same trend can be seen with <b>km_travelled</b>.

Now, let us closely inspect the <b>price_charged</b> column to try find the reason behind the outliers.
<br />


In [ ]:
master_df[['km_travelled', 'price_charged', 'cost_of_trip']]


In [ ]:
master_df[['km_travelled', 'price_charged', 'cost_of_trip']].describe()


### <i>Observations</i>
------------------------------
As you can see the 75th percentile of <b>km_travelled</b>, <price_charged</b> and <b>cost_of_trip</b> are approx. 33, 584 and 414 respectively. This goes with the 25th and 50th percentiles. On the other hand, the maximum value in these columns are 48, 2048, 691 respectively. While <b>klm_travelled</b> and <b>cost_of_trip</b> values seem to be reasonable, there is a huge jump in the value of <b>price charged</b>.

Let us take a closer look at these hyped values in the <b>price_charged</b> column.

<br />



In [ ]:
fig, ax = plt.subplots(1, 3, figsize=(15,5))
sns.boxplot(y=master_df.km_travelled, orient='v', ax=ax[0])
sns.boxplot(y=master_df.price_charged, orient='v', ax=ax[1])
sns.boxplot(y=master_df.cost_of_trip, orient='v', ax=ax[2])


In [ ]:
fig, ax = plt.subplots(1, 3, figsize=(15,5))
sns.distplot(master_df.km_travelled, bins=50, ax=ax[0])
sns.distplot(master_df.price_charged, bins=50, ax=ax[1])
sns.distplot(master_df.cost_of_trip, bins=50, ax=ax[2])


In [ ]:
fig, ax = plt.subplots(1, 3, figsize=(15,5))
sns.scatterplot(master_df.km_travelled, master_df.price_charged, ax=ax[0])
sns.scatterplot(master_df.cost_of_trip, master_df.price_charged, ax=ax[1])
sns.scatterplot(master_df.km_travelled, master_df.cost_of_trip, ax=ax[2])


### <i>Observations</i>
<hr />

As you can see, the distribution of <b>km_travelled</b> and <b>cost_of_trip</b> are some what resemble and can almost be approximated to a trapezoid with 2 right angles.

While that of <b>price_charged</b> is quite different. It follows the same trend as the other two for <b>price_charged < 500</b>. After that the slope is dropping at a faster rate towards large values.
    
<br />

 

In [ ]:
# look at rows with oitliers in price_charged column
q1 = np.percentile(master_df.price_charged, 25, interpolation='midpoint')
q3 = np.percentile(master_df.price_charged, 75, interpolation='midpoint')

IQR = q3 - q1

print(q1, q3, IQR)


In [ ]:
upper = q3 + 1.5*IQR
lower = q1 - 1.5*IQR

print(upper, lower)


In [ ]:
outlier_df = master_df[(master_df.price_charged >= upper) | (master_df.price_charged <= lower)] \
                        [['km_travelled', 'price_charged', 'cost_of_trip']]
outlier_df.shape


In [ ]:
outlier_df.info()

In [ ]:
outlier_df.head()

In [ ]:
print("Percentage of outliers in price_charged: {}%\n\n".format(round(outlier_df.shape[0] / master_df.shape[0] * 100, 2)))


<br />
<br />

### 9. Sanity checks
<hr />


In [ ]:
master_df.describe()

<br />


<br />
<br />
<br />


## Exploratory Data Analysis
<hr />

1. Univariate analysis
2. Bivariate analysis
3. Multivariate analysis
<br />
<br />

### Univariate analysis
<hr />


In [ ]:
master_df.info()

In [ ]:
master_df.head(3)

In [ ]:
numeric_cols = ['km_travelled', 'price_charged', 'cost_of_trip', 'income_usd_per_month', 'population', 'users']
category_cols = ['company', 'city', 'payment_mode', 'gender', 'age', 'age_group']
date_cols = ['month_of_travel', 'year_of_travel', 'day_of_travel', 'weekday_of_travel', 'is_weekend']


In [ ]:
# Analyse the numerical columns
for col in numeric_cols:
    plt.figure(figsize=(10,5))
    sns.histplot(master_df[col])
    plt.xlabel(col, fontsize=15)
    plt.show()
    

<br />


In [ ]:
# Analyse the categorical columns
for col in category_cols:
    plt.figure(figsize=(15,5))
    order_of_values = master_df[col].value_counts().index
    sns.countplot(master_df[col], order=order_of_values)
    plt.xlabel(col, fontsize=15)
    plt.xticks(label=order_of_values, fontsize=12, rotation='vertical' if col=='city' else 'horizontal')
    plt.show()
    

<br />

In [ ]:
# Analyse the date columns
for col in date_cols:
    plt.figure(figsize=(15,5))
    order_of_values = master_df[col].value_counts().index
    sns.countplot(master_df[col], order=order_of_values)
    plt.xlabel(col, fontsize=15)
    plt.xticks(label=order_of_values, fontsize=12)
    plt.show()
    

<br />


In [ ]:
fig, ax = plt.subplots(4,3, figsize=(20,20))

# select numeric columns only
for i, col in enumerate(list(master_df.select_dtypes(include=np.number).columns)):
    r = i // 3
    c = i % 3
    sns.boxplot(master_df[col], ax=ax[r][c])
    ax[r,c].set_title(col, fontsize=15)
    
ax[3,1].set_axis_off()
ax[3,2].set_axis_off()


<br />

### Observations
<ol>
    <li> There is more data pertaining to yellow cab </li>
    <li> More no of cab users are from Newyork (almost double the no of the runner city), followed by Chicago, WashingtonDC and Ls Angeles </li>
    <li> More payment has been made on card than cash </li>
    <li> More male cab users than female </li>
    <li> Most no of cab users fall in the age groups of 20-40 </li>
</ol>        
        
<br />
<br />
        

### Segmented univariate analysis
<hr />

Let us analyse the data by segmenting based on the company, Yellow cab and Pink cab.
<br />


In [ ]:
fig, ax = plt.subplots(2,3, figsize=(20,15))

# select numeric columns only
for i, col in enumerate(numeric_cols):
    r = i // 3
    c = i % 3
    sns.boxplot(x='company', y=col, data=master_df, ax=ax[r][c])
    ax[r,c].set_title(col, fontsize=15)
    

<br />

In [ ]:
# Analyse the categorical columns
for col in category_cols:
    plt.figure(figsize=(15,5))
    order_of_values = master_df[col].value_counts().index
    sns.countplot(x=col, hue='company', data=master_df, order=order_of_values)
    plt.xlabel(col, fontsize=15)
    plt.ylabel('count', fontsize=15)
    plt.xticks(label=order_of_values, fontsize=12, rotation='vertical' if col=='city' else 'horizontal')
    plt.show()
    

<br />

In [ ]:
# Analyse the date columns
for col in date_cols:
    plt.figure(figsize=(15,5))
    order_of_values = master_df[col].value_counts().index
    sns.countplot(x=col, hue='company', data=master_df, order=order_of_values)
    plt.xlabel(col, fontsize=15)
    plt.ylabel('count', fontsize=15)
    plt.xticks(label=order_of_values, fontsize=12, rotation='vertical' if col=='city' else 'horizontal')
    plt.show()
    

<br />

### Observations
<hr />
<ol>
    <li> The km_travelled is comparable for both Yellow and Pink cab </li>
    <li> Both price_charged and cost_of_trip is comparatively higher for Yellow cab than Pink cab </li>
    <li> In all the cities where the no of users is high, more no of poeple have opted for Yellow cab, especially in Neywork where this amount is almost 4 times that for Pink cab </li>
    <li> All age groups have shown more interest in Yellow cab than pink cab </li>
    <li> There is more demand for cabs (here also Yellow cab is in favour) during the last months of the year (highest in december) and low during the first months of the year (lowest in february). </li>
    <li> More people have been found to using cab on Fridays, Saturdays and Sundays .i.e., when the weekend starts and during the weekend. </li>
</ol>        
        
<br />
<br />
        

<br />
<br />

### Bivariate analysis
<hr />


In [ ]:
master_df.info()

In [ ]:
fig, ax = plt.subplots(2,3, figsize=(20,10))

# select numeric columns only
for i, col in enumerate(numeric_cols):
    r = i // 3
    c = i % 3
    sns.boxplot(x='company', y=col, data=master_df, ax=ax[r][c])
    ax[r,c].set_title(col, fontsize=15)

In [ ]:
df = master_df.pivot_table(index='company', columns='month_of_travel', values='income_usd_per_month', aggfunc='sum')
df


In [ ]:
order_of_values = df.sum().sort_values(ascending=False).index
order_of_values


In [ ]:
df.T.plot(kind='bar', ylabel='income_usd_per_month', figsize=(15,5))


<br />


<br />
<br />

### Multivariate analysis
<hr />


In [ ]:
# Pair plot
plt.figure(figsize=(20, 20))
sns.pairplot(master_df)


<br />

### Inference
<hr />
It is best to invest in <b>Yellow Cab</b>. 

The reasons for the same are:
- More income per month 
- More users
- Comparatively higher cost of trip and price charged, but not too high for users to look in for alternatives.

<br />
<br />
        